In [1]:
### 1) Download the DataSet:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("nikhil7280/student-performance-multiple-linear-regression")

print("Path to dataset files:", path)

100%|██████████████████████████████████████| 48.5k/48.5k [00:00<00:00, 3.58MB/s]

Extracting files...
Path to dataset files: /Users/romainkuhne/.cache/kagglehub/datasets/nikhil7280/student-performance-multiple-linear-regression/versions/1


In [4]:
### 2) Get the downloaded DataSet and store it into a pd.DataFrame:
import os
import pandas as pd
import numpy as np
# Path to dataset folder
dataset_path = "/Users/romainkuhne/.cache/kagglehub/datasets/nikhil7280/student-performance-multiple-linear-regression/versions/1"

# Find the CSV file (assuming only one CSV in the folder)
for file_name in os.listdir(dataset_path):
    if file_name.endswith(".csv"):
        csv_file = os.path.join(dataset_path, file_name)
        break

# Load the CSV into a DataFrame
df = pd.read_csv(csv_file)

# Display the DataFrame
print(df)

      Hours Studied  Previous Scores Extracurricular Activities  Sleep Hours  \
0                 7               99                        Yes            9   
1                 4               82                         No            4   
2                 8               51                        Yes            7   
3                 5               52                        Yes            5   
4                 7               75                         No            8   
...             ...              ...                        ...          ...   
9995              1               49                        Yes            4   
9996              7               64                        Yes            8   
9997              6               83                        Yes            8   
9998              9               97                        Yes            7   
9999              7               74                         No            8   

      Sample Question Papers Practiced 

In [ ]:
### Question 1: Predict the performance index of students using Multiple Linear Regression Model:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

async def create_feature_and_prediction_df(df: pd.DataSet) -> tuple[pd.DataSet, pd.DataSet]:
    # 1) Define the FeatureSet:
    X = df.drop(axis=1, subset=["Performance Index"])
    # 2) Define the LabelSet:
    y = df["Performance Index"]
    return X, y

async def gradient_descend(X: pd.DataFrame, y: pd.DataFrame, learningRate=0.01, n_iterations=1000)
